In [1]:
import numpy as np
import altair as alt
import pandas as pd
import database as db

query = """
SELECT p.year, p.day, p.star, p.time as personal, s.time as global, p.position as position
FROM personal p
JOIN scores s
ON p.year = s.year AND p.day = s.day AND p.star = s.star
WHERE s.position = 100
"""

conn = db.open_db('aoc.db')
df = db.do_df(conn, query)
df['personal'] /= 60
df['global'] /= 60
df['relative_time'] = df['personal'] / df['global']
df.value_counts('year')

year
2017    50
2020    50
2021    50
dtype: int64

[]

In [7]:
selection = alt.selection_multi(fields=['year'])

color = alt.condition(selection,
                      alt.Color('year:N', legend=None),
                      alt.value('lightgray'))

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type='log', domain=(1, 400))),
    y=alt.Y('global:Q',scale=alt.Scale(type='log', domain=(1,170))),
    color = color,
    tooltip=['year','day','star']
).properties(
    title='Personal time vs global time',
    width=600,
    height=600)


legend = alt.Chart(df).mark_point().encode(
    y=alt.Y('year:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

chart | legend

alt.HConcatChart(...)

In [46]:
dom = 'linear'
chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type=dom, domain=(1, 400))),
    y=alt.Y('global:Q',scale=alt.Scale(type=dom, domain=(1,170))),
    color = 'year:N',
    tooltip=['year','day','star']
)
chart + chart.transform_regression('personal', 'global',groupby=['year'], method='linear').mark_line()

alt.LayerChart(...)

In [41]:
# the regression didn't really have a good fit
b  = chart.transform_regression('personal', 'global', params=True, groupby=['year'], method='poly').mark_line()
import altair_transform
print(altair_transform.extract_data(b))

C:\Users\Gebruiker\AppData\Roaming\Python\Python310\site-packages\altair_transform\transform\regression.py:190: RuntimeWarning: divide by zero encountered in log
  self._model = Polynomial.fit(np.log(x), y, 1)
C:\Users\Gebruiker\AppData\Roaming\Python\Python310\site-packages\numpy\polynomial\polyutils.py:299: RuntimeWarning: invalid value encountered in double_scalars
  off = (old[1]*new[0] - old[0]*new[1])/oldlen
C:\Users\Gebruiker\AppData\Roaming\Python\Python310\site-packages\numpy\polynomial\polyutils.py:368: RuntimeWarning: invalid value encountered in multiply
  return off + scl*x


LinAlgError: SVD did not converge in Linear Least Squares

In [24]:
# df.groupby('Category').agg({'Item':'size','shop1':['sum','mean','std'],'shop2':['sum','mean','std'],'shop3':['sum','mean','std']})
stats = ['median', 'mean', 'max', 'min']
df.groupby('year').agg({'global':stats, 'personal':stats, 'relative_time':stats, 'position':stats }).T

year                        2017         2020         2021
global        median   10.650000     8.158333     9.300000
              mean     14.418333    11.274000    16.929333
              max      54.683333    73.783333    76.750000
              min       1.883333     2.483333     1.083333
personal      median   26.000000    22.058333    20.058333
              mean     37.600000    57.400000    57.375000
              max     193.000000   402.983333   360.083333
              min       0.000000     4.733333     1.650000
relative_time median    2.211752     2.998484     2.305100
              mean      2.604343     4.251502     2.967341
              max      12.616822    23.079290    13.322075
              min       0.000000     1.107143     1.036827
position      median         NaN  1667.000000  1079.000000
              mean           NaN  1987.180000  1493.860000
              max            NaN  6577.000000  3935.000000
              min            NaN   189.000000   119.000000

In [ ]:

# first puzzle time is field 1
x = np.array([int(s[1]) for s in scores])
y = np.array([int(s[1]) for s in leaderboard_scores])

source = pd.DataFrame({'personal':x,'leaderboard':y,'label':list(range(1,26))})
points_star1 = alt.Chart(source).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(domain=(0, 200))),
    y=alt.Y('leaderboard:Q',scale=alt.Scale(domain=(0,60)))
).properties(
    title='Performance on first star '+str(YEAR),
    width=600,
    height=600)

text_star1 = points_star1.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='label'
)

# total puzzle time is field 2
x = np.array([int(s[2]) for s in scores])
y = np.array([int(s[2]) for s in leaderboard_scores])

source = pd.DataFrame({'personal':x,'leaderboard':y,'label':list(range(1,26))})
points_star2 = alt.Chart(source).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(domain=(0, 200))),
    y=alt.Y('leaderboard:Q',scale=alt.Scale(domain=(0,60)))
).properties(
    title='Performance on both stars '+str(YEAR),
    width=600,
    height=600)

text_star2 = points_star2.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='label'
)
((points_star1 + text_star1) | (points_star2 + text_star2)).save('chart_'+str(YEAR)+'.html')

